![Cabec%CC%A7alho_notebook.png](cabecalho_notebook.png)

# Classificação de Atividade Humana com PCA

Vamos trabalhar com a base da demonstração feita em aula, mas vamos explorar um pouco melhor como é o desempenho da árvore variando o número de componentes principais.

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.tree import DecisionTreeClassifier
from sklearn.decomposition import PCA
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score



filename_features = "./human+activity+recognition+using+smartphones/UCI HAR Dataset/UCI HAR Dataset/features.txt"
filename_labels = "./human+activity+recognition+using+smartphones/UCI HAR Dataset/UCI HAR Dataset/activity_labels.txt"

filename_subtrain = "./human+activity+recognition+using+smartphones/UCI HAR Dataset/UCI HAR Dataset/train/subject_train.txt"
filename_xtrain = "./human+activity+recognition+using+smartphones/UCI HAR Dataset/UCI HAR Dataset/train/X_train.txt"
filename_ytrain = "./human+activity+recognition+using+smartphones/UCI HAR Dataset/UCI HAR Dataset/train/y_train.txt"

filename_subtest = "./human+activity+recognition+using+smartphones/UCI HAR Dataset/UCI HAR Dataset/test/subject_test.txt"
ffilename_xtest = "./human+activity+recognition+using+smartphones/UCI HAR Dataset/UCI HAR Dataset/test/X_test.txt"
filename_ytest = "./human+activity+recognition+using+smartphones/UCI HAR Dataset/UCI HAR Dataset/test/y_test.txt"

features = pd.read_csv(filename_features, header=None, names=['nome_var'], sep="#")
labels = pd.read_csv(filename_labels, delim_whitespace=True, header=None, names=['cod_label', 'label'])
subject_train = pd.read_csv(filename_subtrain, header=None, names=['subject_id'])
subject_test = pd.read_csv(filename_subtest, header=None, names=['subject_id'])

X_train = pd.read_csv(filename_xtrain, delim_whitespace=True, header=None, names=features['nome_var'].tolist())
X_test = pd.read_csv(ffilename_xtest, delim_whitespace=True, header=None, names=features['nome_var'].tolist())

y_train = pd.read_csv(filename_ytrain, header=None, names=['cod_label'])
y_test = pd.read_csv(filename_ytest, header=None, names=['cod_label'])

## PCA com variáveis padronizadas

Reflexão sobre a escala das variáveis:

**Variáveis em métricas muito diferentes** podem interferir na análise de componentes principais. Lembra que variância é informação pra nós? Pois bem, tipicamente se há uma variável monetária como salário, vai ter uma ordem de variabilidade bem maior que número de filhos, tempo de emprego ou qualquer variável dummy. Assim, as variáveis de maior variância tendem a "dominar" a análise. Nesses casos é comum usar a padronização das variáveis.

Faça duas análises de componentes principais para a base do HAR - com e sem padronização e compare:

- A variância explicada por componente
- A variância explicada acumulada por componente
- A variância percentual por componente
- A variância percentual acumulada por componente
- Quantas componentes você escolheria, em cada caso para explicar 90% da variância?

In [2]:
%%time

def padroniza(s):
    if s.std() > 0:
        s = (s - s.mean())/s.std()
    return s

X_train_pad = pd.DataFrame(X_train).apply(padroniza, axis=0)
X_train_pad.head()

CPU times: total: 46.9 ms
Wall time: 198 ms


,1 tBodyAcc-mean()-X,2 tBodyAcc-mean()-Y,3 tBodyAcc-mean()-Z,4 tBodyAcc-std()-X,5 tBodyAcc-std()-Y,6 tBodyAcc-std()-Z,7 tBodyAcc-mad()-X,8 tBodyAcc-mad()-Y,9 tBodyAcc-mad()-Z,10 tBodyAcc-max()-X,...,552 fBodyBodyGyroJerkMag-meanFreq(),553 fBodyBodyGyroJerkMag-skewness(),554 fBodyBodyGyroJerkMag-kurtosis(),"555 angle(tBodyAccMean,gravity)","556 angle(tBodyAccJerkMean),gravityMean)","557 angle(tBodyGyroMean,gravityMean)","558 angle(tBodyGyroJerkMean,gravityMean)","559 angle(X,gravityMean)","560 angle(Y,gravityMean)","561 angle(Z,gravityMean)"
0,0.200628,-0.063678,-0.419600,-0.868755,-0.939377,-0.737479,-0.859758,-0.938955,-0.766385,-0.855978,...,-0.795305,0.025958,-0.276380,-0.360579,0.062935,-0.778374,-0.026079,-0.687172,0.407918,-0.007567
1,0.055944,0.031484,-0.253891,-0.875366,-0.923839,-0.849247,-0.868472,-0.921936,-0.848870,-0.871300,...,0.130605,-0.897296,-0.767938,0.133002,-0.021460,-1.218722,1.484369,-0.694091,0.409089,0.007875
2,0.073510,-0.043414,-0.076289,-0.868980,-0.907698,-0.893724,-0.863078,-0.898793,-0.896640,-0.863264,...,1.152257,-0.260860,-0.438286,-0.377815,0.391949,0.151197,1.704085,-0.702191,0.410260,0.026501
3,0.066691,-0.208407,-0.249695,-0.870566,-0.939959,-0.921743,-0.864445,-0.938060,-0.925216,-0.863264,...,1.112694,0.591005,0.463123,-0.135016,-0.033635,1.037781,-1.002951,-0.701636,0.414622,0.031712
4,0.030467,0.027585,-0.109840,-0.875128,-0.934815,-0.921281,-0.867325,-0.931726,-0.927965,-0.870201,...,-0.149567,-0.138505,-0.240296,0.340383,0.268468,1.125841,-1.276196,-0.700104,0.425434,0.045222


In [3]:
X_test_pad = pd.DataFrame(X_test).apply(padroniza, axis = 0)

In [4]:
%%time
# Primeiramente iremos fazer o PCA para as variáveis não padronizadas
prcomp = PCA().fit(X_train)
variance = prcomp.explained_variance_ #Variancia explicada
variance_cum = prcomp.explained_variance_.cumsum() #Variancia explicada acumulada
variance_pct = prcomp.explained_variance_ratio_# Variancia percentual
variance_pct_cum = variance_pct.cumsum() #Variancia percentual acumulada
n_components_90 = np.argmax(variance_pct_cum >= 0.9) + 1
n_components_90

CPU times: total: 375 ms
Wall time: 446 ms


34

In [5]:
%%time
# Agora, iremos usar as variavéis padronizadas.
prcomp_2 = PCA().fit(X_train_pad)
variance = prcomp_2.explained_variance_ #Variancia explicada
variance_cum = prcomp_2.explained_variance_.cumsum() #Variancia explicada acumulada
variance_pct = prcomp_2.explained_variance_ratio_ # Variancia percentual
variance_pct_cum = prcomp_2.explained_variance_ratio_.cumsum() #Variancia percentual acumulada
n_components_90 = np.argmax(variance_pct_cum >= 0.9) + 1
n_components_90

CPU times: total: 375 ms
Wall time: 470 ms


63

Pelos exemplos acima, vemos que a quantidade de componentes para explicar 90% da variância é maior quando as variáveis estão padronizadas,<br> isso ocorre pois a padronização muda a escala das variáveis, deixando-as com média zero e desvio padrão igual a um.<br> Com isso, a variância se distribui mais uniformemente entre as variáveis, logo, como visto,<br> pode ser necessário um maior número de variáveis para se explicar a mesma variância,<br> já que essa se encontra menos concentrada nos primeiros componentes da PCA.

## Árvore com PCA

Faça duas uma árvore de decisão com 10 componentes principais - uma com base em dados padronizados e outra sem padronizar. Utilize o ```ccp_alpha=0.001```.

Compare a acurácia na base de treino e teste.

In [6]:
pca = PCA(n_components= 10).fit(X_train)
pca_pad = PCA(n_components= 10).fit(X_train_pad)
pc_train = pca.transform(X_train)
pc_test = pca.transform(X_test)
pc_train_pad = pca_pad.transform(X_train_pad)
pc_test_pad = pca_pad.transform(X_test_pad)

In [7]:
%%time
clf = DecisionTreeClassifier(ccp_alpha= 0.01).fit(pc_train, y_train)
clf_pad = DecisionTreeClassifier(ccp_alpha= 0.01).fit(pc_train_pad, y_train)
predict = clf.predict(pc_test)
predict_pad = clf_pad.predict(pc_test_pad)
print(f'Acurácia da árvore não padronizada: {accuracy_score(y_test, predict)}')
print(f'Acurácia da árvore padronizada: {accuracy_score(y_test, predict_pad)}')

Acurácia da árvore não padronizada: 0.8008143875127248
Acurácia da árvore padronizada: 0.7329487614523243
CPU times: total: 93.8 ms
Wall time: 368 ms


Novamente, como foi explicado, a padronização deixa a variância mais homegeneizada entre as variáveis, isso afeta também a acurácia, pois,<br> como vemos, ela é menor em uma árvore com as variáveis padronizadas, sendo que ambas árvores possuem a mesma quantidade de componentes<br> e o mesmo ccp_alpha.